## FAIR USE POLICY
**Please do not leave your Jupyter lab server idle for extended periods of time.** The Jupyter process, active Python kernels, and especially running Spark contexts, claim a minimum amount of cluster resources. These add up and will get starve resources of others eventually. Leaving your environment idle for a few hours (e.g., over lunch) is fine. But letting it idle overnight or for multiple days in which you are not actively using the cluster is not. You can kill the server from your SSH session, by pressing ctrl+c repeatedly, or by selecting *File->Shutdown* from the menu.

### Imports

In [4]:
import os
import time
from datetime import datetime, timedelta, date
import pprint

import collections
import numpy as np
import pandas as pd
import subprocess

import pyspark
# Find Spark
import findspark
findspark.init()


from datetime import datetime, timedelta, date
import datetime as dt
from datetime import timedelta as td
import os
import pandas as pd
import subprocess
import re
import pyarrow as pa
import copy
from pyspark.sql.types import DateType
from string import digits
from dateutil import parser

import nltk
import pyspark
from pyspark.sql import *  
from pyspark.sql.functions import *
import pyspark.sql.functions as F 
from pyspark.sql import functions as F, Window
from pyspark.sql.functions import from_unixtime
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import mean as _mean
from pyspark.sql.window import Window as W
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, LongType
from scipy import stats
from scipy.stats import mannwhitneyu



from functools import reduce
from pyspark.ml.feature import Tokenizer, StopWordsRemover


import json

#### Create Spark Configuration

In [5]:
# Create a SparkConf
APP_NAME = "apwg-var2-app"

spark_conf = pyspark.SparkConf().setAppName(APP_NAME).setMaster("yarn").set(
    "spark.submit.deployMode", "client"
).set("spark.sql.parquet.binaryAsString", "true"
).set("spark.dynamicAllocation.maxExecutors", "16"
).set("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp-spark24_2.11:3.0.0"
).set("spark.sql.debug.maxToStringFields", "1000"
).set("spark.executor.memory", "7G")

#### Start SparkContext
1. This may take a minute to complete
2. You should not (and cannot) start two Spark contexts. If you accidentally run this cell twice or get stuck somehow, restart your Python kernel from the menu above.
3. Please **stop your Spark context** when idling for extended periods of time (see code at bottom of notebook)

In [ ]:
print("[{}] Starting Spark context.".format(datetime.now().replace(microsecond=0)))

# SparkContext
sc = pyspark.SparkContext(conf=spark_conf)

# SQLContext
sqlc = pyspark.SQLContext(sc)

### Your code below

In [4]:

APWG_CLEAN_DATA_CONVERTED_BASE = "PATH TO DATA-RECORDS"

INTEREST_DATE_START = datetime(2017, 12, 18)
INTEREST_DATE_END   = datetime(2019, 8, 16)

# Read JSON files into Spark DF
clean_mails_df = sqlc.read.json(APWG_CLEAN_DATA_CONVERTED_BASE, multiLine=True).withColumn(
    "parsed_date", F.from_unixtime(F.col("date_received")).cast("date")
).filter(
    # Filter date range of interest
    (F.col("parsed_date") >= INTEREST_DATE_START.date().isoformat()) &
    (F.col("parsed_date") <= INTEREST_DATE_END.date().isoformat())
)



In [5]:
#filter emails in english language
original_df = clean_mails_df


In [6]:
original_df = original_df.filter(original_df.language == "english")


In [7]:
h1_df = original_df   
h1_df = h1_df.select('parsed_date', 'id')


In [8]:
remover = StopWordsRemover(inputCol='body_words', outputCol='words_clean')
h2_df = remover.transform(original_df)

h2_df = h2_df.drop("body_words")

word_list=['unsubscribed', 'hack', 'takedown', 'password', 'transparent',\
           'attempt', 'redirect', 'impersonate', 'network', 'obsolete', 'illegal', 'damage', 'edit',\
           'unauthenticated', 'initial', 'survey', 'collect', 'victim', 'detect', 'recharge', 'test',\
           'attachment', 'claim', 'profitable', 'virus', 'fraudulent', 'revalidation', 'link', 'description']

#array_intersect function requires two arrays as arguments, create array from the list of given values:
list_col = F.array(*[F.lit(cl) for cl in word_list])
h2_df = h2_df.filter(F.size(F.array_intersect(F.col("words_clean"), list_col)) > 0)
h2_df = h2_df.select('parsed_date', 'id')


In [9]:
h3_df = original_df.filter(original_df.email_has_attachments == "1")
h3_df = h3_df.select('parsed_date', 'id')


In [10]:
# announcement date of ddos attack

ddos_list = ['2018-01-17', '2018-01-27', '2018-02-08', '2018-03-01', '2018-03-06', '2018-05-14', '2018-05-24', '2018-07-30',\
             '2019-01-16','2019-01-31', '2019-02-23', '2019-03-22', '2019-04-16', '2019-06-12', '2019-08-08']


In [11]:
#partitionBy is used to shuffle data before applying the functions
def hypo_1(df, day):
    
    h1_df1 = (df.filter(f"parsed_date < '{day}' and parsed_date > '{day}' - interval 20 days")
             .withColumn('rn', F.dense_rank().over(Window.orderBy(F.desc('parsed_date'))))
             .filter('rn <= 7')
             .drop('rn')
             .withColumn('count_before', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )

    h1_df2 = (df.filter(f"parsed_date < '{day}' + interval 20 days and parsed_date > '{day}'")
             .withColumn('rn', F.dense_rank().over(Window.orderBy('parsed_date')))
             .filter('rn <= 7')
             .drop('rn')
             .withColumn('count_after', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )


    
    return [h1_df1, h1_df2]




In [12]:
def hypo_2(df, day):
    """
    Example usage: df_list = hypo_2(df, '2017-12-18', 15)
    Returns a list of 2 dataframes.
    """
    h2_df1 = (df.filter(f"parsed_date < '{day}' and parsed_date > '{day}' - interval 20 days")
             .withColumn('rn', F.dense_rank().over(Window.orderBy(F.desc('parsed_date'))))
             .filter('rn <= 15')
             .drop('rn')
             .withColumn('count_before', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )
 
    
    h2_df2 = (df.filter(f"parsed_date < '{day}' + interval 20 days and parsed_date > '{day}'")
             .withColumn('rn', F.dense_rank().over(Window.orderBy('parsed_date')))
             .filter('rn <= 15')
             .drop('rn')
             .withColumn('count_after', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )

    return [h2_df1, h2_df2]

In [13]:
def hypo_3(df, day):
    """
    Example usage: df_list = hypo_3(df, '2017-12-18', 15)
    Returns a list of 2 dataframes.
    """
    h3_df1 = (df.filter(f"parsed_date < '{day}' and parsed_date > '{day}' - interval 20 days")
             .withColumn('rn', F.dense_rank().over(Window.orderBy(F.desc('parsed_date'))))
             .filter('rn <= 15')
             .drop('rn')
             .withColumn('count_before', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )
 
    
    h3_df2 = (df.filter(f"parsed_date < '{day}' + interval 20 days and parsed_date > '{day}'")
             .withColumn('rn', F.dense_rank().over(Window.orderBy('parsed_date')))
             .filter('rn <= 15')
             .drop('rn')
             .withColumn('count_after', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )

    
    
    return [h3_df1, h3_df2]

In [14]:
h1_df1_list = []
h1_df2_list = []

for d in ddos_list:
    h1_df1, h1_df2 = hypo_1(h1_df, d)
    
    h1_df1 = h1_df1.dropDuplicates(['parsed_date', 'count_before'])
    h1_df2 = h1_df2.dropDuplicates(['parsed_date', 'count_after'])
    
    h1_df1 = h1_df1.withColumn("sum_before", F.sum("count_before").over(Window.partitionBy("ddos_date")))
    h1_df2 = h1_df2.withColumn("sum_after", F.sum("count_after").over(Window.partitionBy("ddos_date")))
    
    
    h1_df1_list.append(h1_df1)
    h1_df2_list.append(h1_df2)
    
    
hypo1_df1 = reduce(DataFrame.unionAll, h1_df1_list)
hypo1_df2 = reduce(DataFrame.unionAll, h1_df2_list)


hypo1_df1 = (date_df.join(hypo1_df1, 'ddos_date', 'left')
                 .fillna(0, 'count_before')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_before').alias('sum_before'))
         )
hypo1_df2 = (date_df.join(hypo1_df2, 'ddos_date', 'left')
                 .fillna(0, 'count_after')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_after').alias('sum_after'))
         )

In [15]:
h2_df1_list = []
h2_df2_list = []

for d in ddos_list:
    h2_df1, h2_df2 = hypo_2(h2_df, d)
    
    h2_df1 = h2_df1.dropDuplicates(['parsed_date', 'count_before'])
    h2_df2 = h2_df2.dropDuplicates(['parsed_date', 'count_after'])
    
    h2_df1 = h2_df1.withColumn("sum_before", F.sum("count_before").over(Window.partitionBy("ddos_date")))
    h2_df2 = h2_df2.withColumn("sum_after", F.sum("count_after").over(Window.partitionBy("ddos_date")))
    
    h2_df1_list.append(h2_df1)
    h2_df2_list.append(h2_df2)
    
    
hypo2_df1 = reduce(DataFrame.unionAll, h2_df1_list)
hypo2_df2 = reduce(DataFrame.unionAll, h2_df2_list)

hypo2_df1 = (date_df.join(hypo2_df1, 'ddos_date', 'left')
                 .fillna(0, 'count_before')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_before').alias('sum_before'))
         )
hypo2_df2 = (date_df.join(hypo2_df2, 'ddos_date', 'left')
                 .fillna(0, 'count_after')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_after').alias('sum_after'))
         )

In [16]:
h3_df1_list = []
h3_df2_list = []

for d in ddos_list:
    h3_df1, h3_df2 = hypo_3(h3_df, d)
    
    h3_df1 = h3_df1.dropDuplicates(['parsed_date', 'count_before'])
    h3_df2 = h3_df2.dropDuplicates(['parsed_date', 'count_after'])
    
    h3_df1 = h3_df1.withColumn("sum_before", F.sum("count_before").over(Window.partitionBy("ddos_date")))
    h3_df2 = h3_df2.withColumn("sum_after", F.sum("count_after").over(Window.partitionBy("ddos_date")))
    
    h3_df1_list.append(h3_df1)
    h3_df2_list.append(h3_df2)
    

hypo3_df1 = reduce(DataFrame.unionAll, h3_df1_list)
hypo3_df2 = reduce(DataFrame.unionAll, h3_df2_list)

hypo3_df1 = (date_df.join(hypo3_df1, 'ddos_date', 'left')
                 .fillna(0, 'count_before')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_before').alias('sum_before'))
         )
hypo3_df2 = (date_df.join(hypo3_df2, 'ddos_date', 'left')
                 .fillna(0, 'count_after')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_after').alias('sum_after'))
         )

In [15]:
hypo1_df1.orderBy('ddos_date').show(25)
hypo1_df2.orderBy('ddos_date').show(25)

+-----------+-------+------------+----------+----------+
|parsed_date|     id|count_before| ddos_date|sum_before|
+-----------+-------+------------+----------+----------+
| 2018-01-16|1520036|        1228|2018-01-17|      3084|
| 2018-01-14|1516364|         793|2018-01-17|      3084|
| 2018-01-15|1518451|        1063|2018-01-17|      3084|
| 2018-01-24|1536787|        1301|2018-01-27|      3533|
| 2018-01-26|1539203|        1106|2018-01-27|      3533|
| 2018-01-25|1537091|        1126|2018-01-27|      3533|
| 2019-01-03|2325105|        1250|2019-01-16|      1298|
| 2019-01-04|2327127|          48|2019-01-16|      1298|
+-----------+-------+------------+----------+----------+

+-----------+-------+-----------+----------+---------+
|parsed_date|     id|count_after| ddos_date|sum_after|
+-----------+-------+-----------+----------+---------+
| 2018-01-19|1525745|       1037|2018-01-17|     3252|
| 2018-01-20|1527874|       1045|2018-01-17|     3252|
| 2018-01-18|1523955|       1170|2018-01

#### Stop the SparkContext
note: don't run this block unless you actually want to stop your context

In [21]:
print("[{}] Stopping Spark context.".format(datetime.now().replace(microsecond=0)))
sc.stop()

[2021-04-23 11:41:07] Stopping Spark context.
